<a href="https://colab.research.google.com/github/atrahay/news_nao/blob/main/News_nao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import requests

API_KEY = "258275c0180041fb82b5f9d574799473"

url = f"https://newsapi.org/v2/everything?q=karate&from=2023-02-20&sortBy=publishedAt&apiKey={API_KEY}"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print('Error:', response.status_code)


{'status': 'ok', 'totalResults': 431, 'articles': [{'source': {'id': 'cbc-news', 'name': 'CBC News'}, 'author': 'Taylor Simmons', 'title': 'Calgary teen Emily Vigneault punches her way to Canada Winter Games boxing gold medal', 'description': "Emily Vigneault, 16, won the gold medal in the women's 60 kilogram division at the Canada Winter Games in P.E.I. earlier this month. She’s the inaugural winner in the category, with women’s boxing only being introduced to the event this year.", 'url': 'https://www.cbc.ca/news/canada/calgary/airdrie-teen-boxing-gold-canada-winter-games-1.6782772', 'urlToImage': 'https://i.cbc.ca/1.6782989.1679087012!/fileImage/httpImage/image.JPG_gen/derivatives/16x9_620/emily-vigneault.JPG', 'publishedAt': '2023-03-19T11:00:00Z', 'content': "Now back at her home gym in Airdrie, Alta. Emily Vigneault still can't quite believe she's a gold medallist.\r\nShe took the podium in the women's boxing 60 kilogram division of the Canada Winter Games… [+3997 chars]"}, {'sou

In [22]:
import pandas as pd

df = pd.DataFrame(columns=['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])

df

,source,author,title,description,url,urlToImage,publishedAt,content


In [20]:
data["articles"][0]

{'source': {'id': 'cbc-news', 'name': 'CBC News'},
 'author': 'Taylor Simmons',
 'title': 'Calgary teen Emily Vigneault punches her way to Canada Winter Games boxing gold medal',
 'description': "Emily Vigneault, 16, won the gold medal in the women's 60 kilogram division at the Canada Winter Games in P.E.I. earlier this month. She’s the inaugural winner in the category, with women’s boxing only being introduced to the event this year.",
 'url': 'https://www.cbc.ca/news/canada/calgary/airdrie-teen-boxing-gold-canada-winter-games-1.6782772',
 'urlToImage': 'https://i.cbc.ca/1.6782989.1679087012!/fileImage/httpImage/image.JPG_gen/derivatives/16x9_620/emily-vigneault.JPG',
 'publishedAt': '2023-03-19T11:00:00Z',
 'content': "Now back at her home gym in Airdrie, Alta. Emily Vigneault still can't quite believe she's a gold medallist.\r\nShe took the podium in the women's boxing 60 kilogram division of the Canada Winter Games… [+3997 chars]"}

In [35]:
my_dict = data["articles"][0]

my_list = [my_dict['source']['name'], my_dict['author'], my_dict['title'], my_dict['description'], my_dict['url'], my_dict['urlToImage'], my_dict['publishedAt'], my_dict['content']]

#df = df.loc(my_list)
df.loc[0]=my_list

df


,source,author,title,description,url,urlToImage,publishedAt,content
0,CBC News,Taylor Simmons,Calgary teen Emily Vigneault punches her way t...,"Emily Vigneault, 16, won the gold medal in the...",https://www.cbc.ca/news/canada/calgary/airdrie...,https://i.cbc.ca/1.6782989.1679087012!/fileIma...,2023-03-19T11:00:00Z,"Now back at her home gym in Airdrie, Alta. Emi..."


In [36]:
for counter in range(100):

  my_dict = data["articles"][counter]

  my_list = [my_dict['source']['name'], my_dict['author'], my_dict['title'], my_dict['description'], my_dict['url'], my_dict['urlToImage'], my_dict['publishedAt'], my_dict['content']]

  df.loc[counter]=my_list

df

,source,author,title,description,url,urlToImage,publishedAt,content
0,CBC News,Taylor Simmons,Calgary teen Emily Vigneault punches her way t...,"Emily Vigneault, 16, won the gold medal in the...",https://www.cbc.ca/news/canada/calgary/airdrie...,https://i.cbc.ca/1.6782989.1679087012!/fileIma...,2023-03-19T11:00:00Z,"Now back at her home gym in Airdrie, Alta. Emi..."
1,Vnexpress.net,VnExpress,Karate Việt Nam thống trị giải Đông Nam Á 2023,Philippines- Karate Việt Nam về nhất với 23 HC...,https://vnexpress.net/karate-viet-nam-thong-tr...,https://vcdn1-thethao.vnecdn.net/2023/03/19/ka...,2023-03-19T10:37:28Z,"PhilippinesKarate Vit Nam v nht vi 23 HC vàng,..."
2,NDTV News,None,Nagaland Minister's Video Of Cloud-Capped Moun...,The beautiful video opens to show a picturesqu...,https://www.ndtv.com/offbeat/nagaland-minister...,https://c.ndtvimg.com/2023-03/9rp0fojo_mountai...,2023-03-19T09:17:12Z,Internet users loved the stunning clip\r\nTemj...
3,USA Today,Matthew Wells,UFC 286 results: Gunnar Nelson dominates on th...,Gunnar Nelson showed off his slick submission ...,https://mmajunkie.usatoday.com/2023/03/ufc-286...,https://mmajunkie.usatoday.com/wp-content/uplo...,2023-03-18T22:46:04Z,"After a year away from the octagon, UFC welter..."
4,MMA Mania,Dan Hiergesell,Highlights! ‘Gunni’ Ends ‘Bam Bam’ With Early Sub,Watch UFC 286 full fight video highlights: Gun...,https://www.mmamania.com/2023/3/18/23646202/wa...,https://cdn.vox-cdn.com/thumbor/hzbb9UOdhHzEdL...,2023-03-18T22:40:37Z,Gunnar Nelson scored his first submission fini...
...,...,...,...,...,...,...,...,...
95,PBS,"Terry Tang, Associated Press","Bruce Lee, Anna May Wong heirs talk legacy, ro...","Bruce Lee's daughter, Shannon Lee, and Anna Ma...",https://www.pbs.org/newshour/arts/bruce-lee-an...,https://d3i6fh83elv35t.cloudfront.net/newshour...,2023-03-10T22:33:48Z,Almost every working Asian actor in Hollywood ...
96,Daily Mail,Cole Moreton,ANNEKA RICE: I'm not a very sexy person,"Ahead of her return to screens, Anneka Rice, 6...",https://www.dailymail.co.uk/femail/article-118...,https://i.dailymail.co.uk/1s/2023/03/07/17/684...,2023-03-10T22:31:40Z,She's back! Anneka Rice is putting on a jumpsu...
97,CBS Sports,Shakiel Mahjouri,Combate Global -- Alejandro Gavidia vs. Justin...,Combate Global makes its 2023 debut with a Mex...,https://www.cbssports.com/mma/news/combate-glo...,https://sportshub.cbsistatic.com/i/r/2021/07/1...,2023-03-10T20:33:51Z,Combate Global plays host to Mexico vs. USA in...
98,Marca,marca.com,Chuck Norris net worth: What is the fortune of...,"At 82 years old, Chuck Norris has truly done i...",https://www.marca.com/en/lifestyle/celebrity-n...,https://phantom-marca.unidadeditorial.es/69140...,2023-03-10T17:55:03Z,"At 82 years old, Chuck Norris has truly done i..."
